In [6]:
!pip install plotly

  Running setup.py bdist_wheel for retrying: started
  Running setup.py bdist_wheel for retrying: finished with status 'done'
  Stored in directory: C:\Users\cuiti\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


distributed 1.21.8 requires msgpack, which is not installed.
grin 1.2.1 requires argparse>=1.1, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
import re
import warnings
import folium
from folium.plugins import MarkerCluster,HeatMap
from IPython.display import HTML,display
import gmaps
import os
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='ayuan0625', api_key='OVSXti6gso1BEEp6pQfU')
warnings.filterwarnings('ignore')
%pylab inline
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df=pd.read_csv('gun_cleaned2.csv')
map_data=df[['date','city_or_county','address','latitude','longitude','n_killed','n_injured','n_victim','n_suspect','n_victim_Killed','n_victim_Injured','incident_characteristics']]
city_pop=pd.read_csv('city_pop.csv')

In [3]:
#merge the gun violence data with Texas population data
map_pop=map_data.merge(city_pop,
                      left_on='city_or_county',
                      right_on='PLACE').drop(['city_or_county','7/1/2010','7/1/2011','7/1/2012'],1)
map_pop.columns=['date','address','latitude','longitude','n_killed','n_injured','n_victim','n_suspect','n_victim_killed','n_victim_injured','incident_characteristics','PLACE','2013pop','2014pop','2015pop','2016pop','2017pop']

In [4]:
#turn location columns into float type
map_pop=map_pop.dropna(subset=['latitude','longitude','address'])
map_pop['latitude']=map_pop['latitude'].map(lambda x: float(x))
map_pop['longitude']=map_pop['longitude'].map(lambda x: float(x))

In [5]:
#calculate per 1k data in each city over the period from 2013 to 2018
map_pop['AVGpop']=(map_pop['2013pop']+map_pop['2014pop']+map_pop['2015pop']+map_pop['2016pop']+map_pop['2017pop'])/5
city_avgpop=map_pop.set_index('PLACE')['AVGpop'].drop_duplicates()

vio_count=map_pop.groupby('PLACE')['PLACE'].count()
victim_count=map_pop.groupby('PLACE')['n_victim'].sum()
injure_count=map_pop.groupby('PLACE')['n_victim_injured'].sum()
killed_count=map_pop.groupby('PLACE')['n_victim_killed'].sum()

per1k_inj=pd.DataFrame(1.0*injure_count/(city_avgpop/1000)).reset_index()
per1k_inj.columns=['PLACE','per1k_inj']
per1k_kill=pd.DataFrame(1.0*killed_count/(city_avgpop/1000)).reset_index()
per1k_kill.columns=['PLACE','per1k_kill']

per1k_vio=pd.DataFrame(1.0*vio_count/(city_avgpop/1000))
per1k_vio['Total_vio']=vio_count
per1k_vio=per1k_vio.reset_index().merge(city_avgpop.reset_index(),left_on='PLACE',right_on='PLACE')
per1k_vio.columns=['PLACE','per1k_vio','Total_vio','AVGpop']

per1k_vic=pd.DataFrame(1.0*victim_count/(city_avgpop/1000))
per1k_vic['Total_vic']=victim_count
per1k_vic=per1k_vic.reset_index().merge(city_avgpop.reset_index(),left_on='PLACE',right_on='PLACE')
per1k_vic.columns=['PLACE','per1k_vic','Total_vic','AVGpop']


In [6]:
#sort data according to date
map_pop['date']=pd.to_datetime(map_pop['date'])
map_pop=map_pop.set_index('date').sort_index()

In [7]:
#split data into each year
map_2013=map_pop.loc['2013-01-01':'2013-12-31']
map_2014=map_pop.loc['2014-01-01':'2014-12-31']
map_2015=map_pop.loc['2015-01-01':'2015-12-31']
map_2016=map_pop.loc['2016-01-01':'2016-12-31']
map_2017=map_pop.loc['2017-01-01':'2017-12-31']
map_2018=map_pop.loc['2018-01-01':'2018-12-31']

In [8]:
#select data of austin
map_aus=map_pop[map_pop['PLACE']=='Austin']

### 1. Incidents by city

#### Total no. of incidents by city (with map)

In [9]:
hmap=folium.Map(location=(31.45,-99.32), zoom_start=6)
hm=HeatMap(zip(map_pop['latitude'].values, map_pop['longitude'].values),radius=10,blur=20,max_zoom=1)
hmap.add_child(hm)
hmap.save('hmap.html')
hmap

In [10]:
data=per1k_vio[['PLACE','Total_vio']].sort_values(by='Total_vio',ascending=False)
data=data[:20]
data=[go.Bar(x=data['PLACE'],y=data['Total_vio'],opacity=0.7,text=data['Total_vio'], textposition = 'auto',
             marker=dict(color='rgb(220,220,220)',line=dict(color='rgb(105,105,105)',width=1.5),),)]
fig = go.Figure(data=data)
py.iplot(fig, filename='bar1')

#### Incidents per 1k people by city

In [11]:
data=per1k_vio[['PLACE','per1k_vio']].sort_values(by='per1k_vio',ascending=False)
data=data[:20]
data=[go.Bar(x=data['PLACE'],y=data['per1k_vio'],opacity=0.7,text=data['per1k_vio'], textposition = 'auto',
             marker=dict(color='rgb(220,220,220)',line=dict(color='rgb(105,105,105)',width=1.5),),)]
fig = go.Figure(data=data)
py.iplot(fig, filename='bar1')

#### Austin in specific 

In [12]:
hmapaus=folium.Map(location=(30.3,-97.7), zoom_start=11)
hm1=HeatMap(zip(map_aus['latitude'].values, map_aus['longitude'].values),radius=8.5,blur=16,max_zoom=1)
hmapaus.add_child(hm1)
hmapaus.save('hmapaus.html')
hmapaus

### 2. Victims by city

#### Total no. of victims killed and injured by city (rank by total loss)

In [13]:
per1k_ink =per1k_inj.merge(per1k_kill,left_on='PLACE',right_on='PLACE')
data=per1k_vic.merge(per1k_ink,left_on='PLACE',right_on='PLACE')
data=data.merge(per1k_vio,left_on='PLACE',right_on='PLACE').set_index('PLACE')
data['n_injured']=injure_count
data['n_killed']=killed_count
data['total_loss']=data['n_injured']+data['n_killed']
data['total_loss_1k']=data['per1k_inj']+data['per1k_kill']
data=data.reset_index()
dt=data.sort_values(by='total_loss',ascending=False)[:20]
dt1=data.sort_values(by='Total_vio',ascending=False)[:20]
dt2=data.sort_values(by='total_loss_1k',ascending=False)[:20]
dt3=data.sort_values(by='per1k_vio',ascending=False)[:20]

In [14]:
trace1= go.Bar(
    name='victim injured',
    x=dt['PLACE'],y=dt['n_injured'],
    text=dt['n_injured'],
    textposition = 'auto',
    marker=dict(color='rgb(176,224,230)',line=dict(color='rgb(105,105,105)',width=1.5),),
    opacity=0.7
)

trace2= go.Bar(
    name='victim killed',
    x=dt['PLACE'],y=dt['n_killed'],
    text=dt['n_killed'],
    textposition = 'auto',
    marker=dict(color='rgb(250,128,114)',line=dict(color='rgb(105,105,105)',width=1.5),),
    opacity=0.7
)

bar= [trace1,trace2]

py.iplot(bar, filename='bar2')

#### Total no. of victims killed and injured by city (rank by total violence)

In [20]:
trace1= go.Bar(
    name='victim injured',
    x=dt1['PLACE'],y=dt1['n_injured'],
    text=dt1['n_injured'],
    textposition = 'auto',
    marker=dict(color='rgb(176,224,230)',line=dict(color='rgb(105,105,105)',width=1.5),),
    opacity=0.7
)

trace2= go.Bar(
    name='victim killed',
    x=dt1['PLACE'],y=dt1['n_killed'],
    text=dt1['n_killed'],
    textposition = 'auto',
    marker=dict(color='rgb(250,128,114)',line=dict(color='rgb(105,105,105)',width=1.5),),
    opacity=0.7
)

bar= [trace1,trace2]

py.iplot(bar, filename='bar2')

In [16]:
print dt[['PLACE','total_loss']]
print dt1[['PLACE','Total_vio']]

              PLACE  total_loss
253         Houston        1930
402     San Antonio        1199
168          Dallas         773
208      Fort Worth         373
26           Austin         247
160  Corpus Christi         203
311         Lubbock         174
281         Killeen         130
11         Amarillo         126
21        Arlington         120
191         El Paso         107
33         Beaumont          81
307        Longview          80
462            Waco          79
454           Tyler          54
1           Abilene          50
296          Laredo          49
354          Odessa          47
446          Temple          42
255          Humble          40
              PLACE  Total_vio
253         Houston       2303
402     San Antonio       1434
168          Dallas       1056
160  Corpus Christi        545
26           Austin        477
208      Fort Worth        374
11         Amarillo        290
311         Lubbock        275
281         Killeen        235
191         El Pas

#### No. of victims killed and injured per 1k people by city (rank by total loss per 1k people)

In [17]:
trace1= go.Bar(
    name='victim injured per 1k people',
    x=dt2['PLACE'],y=dt2['per1k_inj'],
    text=dt2['per1k_inj'],
    textposition = 'auto',
    marker=dict(color='rgb(176,224,230)',line=dict(color='rgb(105,105,105)',width=1.5),),
    opacity=0.7
)

trace2= go.Bar(
    name='victim killed per 1k people',
    x=dt2['PLACE'],y=dt2['per1k_kill'],
    text=dt2['per1k_kill'],
    textposition = 'auto',
    marker=dict(color='rgb(250,128,114)',line=dict(color='rgb(105,105,105)',width=1.5),),
    opacity=0.7
)

bar= [trace1,trace2]

py.iplot(bar, filename='bar2')

#### No. of victims killed and injured per 1k people by city (rank by total violence per 1k people)

In [18]:
trace1= go.Bar(
    name='victim injured per 1k people',
    x=dt3['PLACE'],y=dt3['per1k_inj'],
    text=dt3['per1k_inj'],
    textposition = 'auto',
    marker=dict(color='rgb(176,224,230)',line=dict(color='rgb(105,105,105)',width=1.5),),
    opacity=0.7
)

trace2= go.Bar(
    name='victim killed per 1k people',
    x=dt3['PLACE'],y=dt3['per1k_kill'],
    text=dt3['per1k_kill'],
    textposition = 'auto',
    marker=dict(color='rgb(250,128,114)',line=dict(color='rgb(105,105,105)',width=1.5),),
    opacity=0.7
)

bar= [trace1,trace2]

py.iplot(bar, filename='bar2')

In [19]:
print dt2[['PLACE','total_loss_1k']]
print dt3[['PLACE','per1k_vio']]

              PLACE  total_loss_1k
154      Colmesneil       6.791171
14         Anderson       4.409171
36           Bedias       4.407228
387            Rice       4.197272
345          Naples       2.995806
39         Bellevue       2.868617
0            Abbott       2.801120
194       Elmendorf       2.684131
276       Kendleton       2.570694
255          Humble       2.541038
339      Montgomery       2.444988
79         Chandler       2.415959
241         Hawkins       2.339729
151      Coldspring       2.228661
270            Katy       2.057389
442           Talco       1.963094
12             Ames       1.889645
326            Maud       1.875821
64          Calvert       1.749169
254  Hughes Springs       1.721961
            PLACE  per1k_vio
339    Montgomery   7.334963
14       Anderson   4.409171
36         Bedias   4.407228
482       Zavalla   4.187605
202  Fayetteville   3.831418
194     Elmendorf   3.757784
154    Colmesneil   3.395586
162      Coupland   3.306878
362 